### Multi-Class Classification
Now that we've explored a binary classification problem with linear and non-linear architectures, we now want to shift to a multi-class problem where there are more than two options that the model needs to be able to classify.

The multi-class data will be artificial data from the scikit-learn `make_blobs()` function. The general flow is as follows:
1. Make the artificial data and convert to tensors
2. Visualize the data
3. Define the model architecture
4. Train the model
5. Adjust hyperparameters as necessary

In [ ]:
# First, lets explore the make_blobs() function. According to the documentation,
# make_blobs is designed for creating artificial multiclass data by creating isotropic, Gaussian clusters
# of points. The data is quite literally "blobs" of points around a "center" in R^n space. The classes could be based 
# on the number of centers in a feature set.

import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import torch

n_points = 50
X_blob, y_blob = make_blobs([n_points, n_points], # array length = num blobs, n_points = points per blob
                            n_features=3,
                            centers=None, random_state=42) # returns coordinates of points (X) and its blob membership

fig = plt.figure()
plt.title('Two blobs with 3 features (x, y, z)')
ax = fig.add_subplot(projection='3d')
ax.scatter(X_blob[:,0], X_blob[:,1], X_blob[:, 2], c=y_blob)

In [ ]:
# Now lets standardize this a little for the actual model. Will define the const values that will be used when creating
# the architecture (allow things to be updated once). This could probably eventually be refactored into a dataclass.

NUM_CLASSES = 4 # This is self explanatory, the number of blobs per training data instance
CLUSTER_POINTS = 100 # This is the number of points that are in each blob
NUM_FEATURES = 2 # This refers to the dimension of the data. In this case, the dimension of the the points in the blobs (the above example is 3D)
CLUSTER_STD_DEV = 1.0 # This changes the spread in each blob (makes classification more difficult!)
RANDOM_SEED = 42
TRAIN_TEST_RATIO = 0.2

In [ ]:
# Now lets create our training data and move to tensors
X_blob, y_blob = make_blobs([CLUSTER_POINTS for centers in range(NUM_CLASSES)],
                            n_features=NUM_FEATURES,
                            centers=None,
                            random_state=RANDOM_SEED,
                            cluster_std=CLUSTER_STD_DEV)

X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.float)

X_train, X_test, y_train, y_test = train_test_split(
    X_blob,
    y_blob,
    test_size=TRAIN_TEST_RATIO, # Ratio of test data to use from full dataset; Training is the complement
    random_state=RANDOM_SEED,
)

In [ ]:
# Now lets inspect our dataset to make sure it looks as expected
print(X_train[0:9]) # expect coordinates from R^2
print(y_train[0:14]) # expect values from 0-3
print(f'X ratio: {len(X_test)/len(X_train)}, y ratio: {len(y_test)/len(y_train)}') # should be ~0.2
for obj in [X_train, X_test, y_train, y_test]: # expecting all to be torch.float
    print(obj.dtype)


In [ ]:
# Now that the dataset properties look good, lets visualize it!
fig = plt.figure()
base_title = f'{NUM_CLASSES} blobs with {NUM_FEATURES} features'
if NUM_FEATURES >= 3:
    plt.title(base_title + ' (first 3 dims.)')
    ax = fig.add_subplot(projection='3d')
    ax.scatter(X_train[:,0], X_train[:,1], X_train[:, 2], c=y_train)
elif NUM_FEATURES == 2:
    plt.title(base_title)
    plt.scatter(X_train[:,0], X_train[:,1], c=y_train)   